In [14]:
def NAF_encode(number):
    """
    Returns NAF representation
    of given integer 'number'
    """
    naf = []
    while number > 0:
        if number & 1:
            temp = 2 - (number & 3)
        else:
            temp = 0
        number = (number - temp) >> 1
        naf.append(temp)
    return naf

class MemoryImage:
    """
    This class will represent a memory image that contains
    a secret key 'a' and the corresponding 'NAF(a)'.
    The class contains a function to corrupt the memory for
    given error rates 'alpha' and 'beta'.
    Alpha is the probability of a 0 bit flipping to 1
    Beta is the probability of a 1 bit flipping to 0
    """
    def __init__(self, a, bitlen, naflen):
        self.a = a
        self.bitlen = bitlen
        self.naflen = naflen
        self.binary = self.int_to_binary(a)
        self.naf = self.naf_to_binary(NAF_encode(a))
        
    def int_to_binary(self, a):
        """
        Returns binary representation of given
        integer 'a'. Prepend 0 bits if 'self.bitlen'
        is greater than the bitlen of 'a'
        """
        binary = []
        while a > 0:
            binary.append(a & 1)
            a = a >> 1
        for x in range(self.bitlen - len(binary)):
            binary.append(0)
        return binary
    
    def get_int(self):
        """
        Returns integer representation
        of self.binary
        """
        res = 0
        for index in range(len(self.binary)):
            if self.binary[index] == 1:
                res = res + 2^index
        return res
    
    def naf_to_binary(self, naf):
        """
        Returns the binary representation
        of the given NAF. Prepends zeroes
        if the length of the given NAF is
        smaller than 'self.naflen'
        """
        for x in range(self.naflen - len(naf)):
            naf.append(0)
        binary_naf = []
        for element in naf:
            if element == 0:
                for x in range(8):
                    binary_naf.append(0)
            elif element == 1:
                binary_naf.append(1)
                for x in range(7):
                    binary_naf.append(0)
            elif element == -1:
                for x in range(8):
                    binary_naf.append(1)
        return binary_naf
    
    def corrupt_memory(self, alpha, beta):
        """
        Randomly flips bits in 'self.binary'
        and 'self.naf' depending on given
        error rates 'alpha' and 'beta'
        """
        alpha = alpha * 1000
        beta  = beta * 1000
        for index in range(len(self.binary)):
            if self.binary[index] == 0:
                if ZZ.random_element(1000) < alpha:
                    self.binary[index] = 1
            elif self.binary[index] == 1:
                if ZZ.random_element(1000) < beta:
                    self.binary[index] = 0
        for index in range(len(self.naf)):
            if self.naf[index] == 0:
                if ZZ.random_element(1000) < alpha:
                    self.naf[index] = 1
            elif self.naf[index] == 1:
                if ZZ.random_element(1000) < beta:
                    self.naf[index] = 0
    
    def correlate(self, noisy_mem, alpha, beta, C, start_bit, start_naf, length):
        """
        Correlate test as described by Poettering and Sibborn in
        'Cold Boot Attacks in the Discrete Logarithm Setting'
        Arguments:
            noisy_mem: noisy memory image to compare partial solution to
            alpha: Pr(0 -> 1)
            beta : Pr(1 -> 0)
            C: treshold
            start_bit: bit at which the correlate test starts
            start_naf: naf digit at which the correlate test starts
            length: number of naf digits that are considered by correlate
        """
        # This is the 'end_bit'
        binary_bit_len = len(self.binary)
        
        # n_00 := number of bits that are 0 in s_i and r
        # n_01 := number of bits that are 0 in s_i and 1 in r
        # n_10 := number of bits that are 1 in s_i and 0 in r
        # n_11 := number of bits that are 1 in s_i and r
        n_00 = 0
        n_01 = 0
        n_10 = 0
        n_11 = 0
        
        for index in range(start_bit, binary_bit_len):
            if self.binary[index] == 0 and noisy_mem.binary[index] == 0:
                n_00 = n_00 + 1
            elif self.binary[index] == 0 and noisy_mem.binary[index] == 1:
                n_01 = n_01 + 1
            elif self.binary[index] == 1 and noisy_mem.binary[index] == 0:
                n_10 = n_10 + 1
            elif self.binary[index] == 1 and noisy_mem.binary[index] == 1:
                n_11 = n_11 + 1
        for index in range(start_naf*8, (start_naf+length)*8):
            if self.naf[index] == 0 and noisy_mem.naf[index] == 0:
                n_00 = n_00 + 1
            elif self.naf[index] == 0 and noisy_mem.naf[index] == 1:
                n_01 = n_01 + 1
            elif self.naf[index] == 1 and noisy_mem.naf[index] == 0:
                n_10 = n_10 + 1
            elif self.naf[index] == 1 and noisy_mem.naf[index] == 1:
                n_11 = n_11 + 1
        
        # n_0 := number of 0s in s_i
        # n_1 := number of 1s in s_i
        n_0 = n_00 + n_01
        n_1 = n_10 + n_11
        
        # c_0 := multinomial test statistic for Correlate^0
        # c_1 := multinomial test statistic for Correlate^1
        c_0 = 0
        c_00 = 0
        c_01 = 0
        c_1 = 0
        c_10 = 0
        c_11 = 0
        
        if n_00 > 0:
            c_00 = - 2*n_00*ln(n_0*(1 - alpha)/n_00)
        if n_01 > 0:
            c_01 = - 2*n_01*ln(n_0*alpha/n_01)
        c_0 = c_00 + c_01
        
        if n_11 > 0:
            c_11 = - 2*n_11*ln(n_1*(1 - beta)/n_11)
        if n_10 > 0:
            c_10 = - 2*n_10*ln(n_1*beta/n_10)
        c_1 = c_11 + c_10
        
        # Test passes if Correlate^0 < C and Correlate^1 < C
        if (c_0 < C) and (c_1 < C):
            return True
        return False

def cba_naf(noisy_mem, t, C, alpha, beta):
    """
    Generic key recovery algorithm for keys with their NAF
    as redundancy as proposed by Poettering and Sibborn
    Arguments:
        noisy_mem: noisy memory image including noisy key
        and noisy NAF
        t: step size
        C: threshold
        alpha: Pr(0 -> 1)
        beta : Pr(1 -> 0)
    """
    cand_set = []
    bitposition = 0
    
    # Stage 1
    for x in range(2^(t+1)):
        part_mem = MemoryImage(x, t + 1, t)
        if part_mem.correlate(noisy_mem, alpha, beta, C, bitposition, 0, t):
            cand_set.append(x)

    bitposition = t + 1
    
    # Stage 2
    for part in range(1, (noisy_mem.bitlen - 1)/t):
        temp_cand_set = []
        for cand in cand_set:
            for x in range(0, 2^(bitposition + t), 2^bitposition):
                part_mem = MemoryImage(x + cand, bitposition+t, bitposition+t)
                if part_mem.correlate(noisy_mem, alpha, beta, C, bitposition, part*t, t):
                    temp_cand_set.append(x + cand)
        bitposition = bitposition + t
        cand_set = temp_cand_set
    
    # End stage
    temp_cand_set = []
    for cand in cand_set:
        for x in range(0, 2^(bitposition + ((noisy_mem.bitlen - 1) % t)), 2^bitposition):
            temp_cand_set.append(x + cand)
            # No need to do ECC arithmetic here since we can simply compare
            # to the original key inside the simulation
    return temp_cand_set

def cba_brute_force(original_mem, noisy_mem, alpha, beta, k=sqrt(2)):
    """
    Bruteforce key recovery for bidirectional errors
    Returns the secret key if the number of errors in
    the erroneous key is smaller than 'max_num_bits_flipped'
    Arguments:
        original_mem: Memory image including secret key without errors
        noisy_mem: noisy memory image including noisy key
        alpha: Pr(0 -> 1)
        beta : Pr(1 -> 0)
        k: variable parameter that determines runtime and success rate
    """
    delta = (alpha + beta) / 2
    n = len(noisy_mem.binary)
    
    expected = n * delta
    variance = expected * (1-delta)

    max_num_bits_flipped = ceil(expected + k * variance)
    
    for j in range(max_num_bits_flipped + 1):
        for lst in Combinations(range(n), j):
            cand = MemoryImage(noisy_mem.get_int(), n, 0)
            for index in lst:
                cand.binary[index] = 1 - cand.binary[index]
            # No need to do ECC arithmetic here since we can simply compare
            # to the original key inside the simulation
            if cand.binary == original_mem.binary:
                return cand.get_int()
    return None

alpha = 0.001
beta = 0.01
t = 7
C = 6
bitlen = 160

num_tries = 100

numbers = []
for a in range(num_tries):
    x = 0

    while x < 2^(bitlen-1):
        x = ZZ.random_element(2^bitlen)
    numbers.append(x)

print("Start Poetterings and Sibborns key recovery")

ctr = 0
for index in range(num_tries):
    x = numbers[index]
    naf = NAF_encode(x)
    orig_mem = MemoryImage(x, bitlen, len(naf))
    corr_mem = MemoryImage(x, bitlen, len(naf))

    corr_mem.corrupt_memory(alpha, beta)

    result = cba_naf(corr_mem, t, C, alpha, beta)

    if x in result:
        ctr = ctr + 1
        print("%X" % x)
    else:
        print("!%X" % x)

print("DONE")
print("Found %d out of %d Keys" % (ctr, num_tries))

print("Start brute force recovery")

ctr = 0
for index in range(num_tries):
    x = numbers[index]
    original_mem = MemoryImage(x, bitlen, 0)
    corr_mem = MemoryImage(x, bitlen, 0)
    corr_mem.corrupt_memory(alpha, beta)

    result = cba_brute_force(original_mem, corr_mem, alpha, beta)

    if result == x:
        ctr = ctr + 1
        print("%X" % x)
    else:
        print("!%X" % x)

print("DONE")
print("Found %d out of %d Keys" % (ctr, num_tries))

Start Poetterings and Sibborns key recovery


KeyboardInterrupt: 